In [1]:
%serialconnect

Connecting to Serial /dev/ttyUSB2 baud=115200 
Ready.



In [2]:
%sendtofile --quiet --source ../basicsockets/connecthotspot.py connecthotspot.py
%sendtofile --quiet --source ../basicsockets/connectwifi.py connectwifi.py
%sendtofile --quiet --source ../wificodes.txt wificodes.txt

Sent 9 lines (327 bytes).
Sent 21 lines (645 bytes).
Sent 1 lines (16 bytes).


In [3]:
import connectwifi
try:
    import uasyncio as asyncio
except:
    import upip
    upip.install('micropython-uasyncio')
    import uasyncio as asyncio


W (1199706) phy_init: failed to load RF calibration data (0x1102), falling back to full calibration
Scanning wifis: BV6000
I (1202296) network: event 1
Connecting to strongest known wifi signal BV6000
> auth (b0)
> assoc (0)
> run (10)
I (1203716) network: event 4
.
I (1206776) network: GOT_IP
Device has ipnumber 192.168.43.6
Installing to: /lib/
Installing micropython-uasyncio 1.2.4 from https://pypi.python.org/packages/49/91/3228f62e1d7238d416b37a04cba1427fb8b4abe3666aff6d8bd1f68cd0ad/micropython-uasyncio-1.2.4.tar.gz
Installing micropython-uasyncio.core 1.5.1 from https://pypi.python.org/packages/22/ce/1e7ef36446d45f3f74d3f955ddd2dc4e6c526c4cd69050a4ee75a65b251d/micropython-uasyncio.core-1.5.1.tar.gz
.

In [17]:
%sendtofile --binary --source ../sensor_websocket_graph/raphael.min.js raphael.min.js


Sent 93251 bytes in 3109 chunks.


In [5]:
%sendtofile --quiet --source ../async_web_serve/webserve_funcs.py     webserve_funcs.py
%sendtofile --quiet --source ../async_web_serve/websocket_funcs.py    websocket_funcs.py
%sendtofile --quiet --source ../sensor_websocket_graph/graphtest.html graphtest.html


Sent 105 lines (3521 bytes).
Sent 72 lines (2538 bytes).
Sent 209 lines (6077 bytes).


In [35]:
%sendtofile --quiet --source ../essential_sensor_code/vl6180_funcs.py vl6180_funcs.py
%sendtofile --quiet --source ../essential_sensor_code/ms5611_funcs.py ms5611_funcs.py


Sent 89 lines (2849 bytes).
Sent 65 lines (1934 bytes).


In [ ]:
%sendtofile --quiet --source ../sensor_websocket_graph/graphtest.html graphtest.html


In [86]:
%serialconnect

attempt to exit past mode
[\r\x03\x02] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x7f15fb59c358, open=True>(port='/dev/ttyUSB2', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to Serial /dev/ttyUSB2 baud=115200 
ets Jun  8 2016 00:22:57

rst:0x1 (POWERON_RESET),boot:0x17 (SPI_FAST_FLASH_BOOT)
configsip: 0, SPIWP:0xee
clk_drv:0x00,q_drv:0x00,d_drv:0x00,cs0_drv:0x00,hd_drv:0x00,wp_drv:0x00
mode:DIO, clock div:2
load:0x3fff0018,len:4
load:0x3fff001c,len:4332
load:0x40078000,len:0
load:0x40078000,len:10992
entry 0x4007a6c4
W (64) rtc_clk: Potentially bogus XTAL frequency: 35 MHz, guessing 40 MHz
OSError: [Errno 2] ENOENT
MicroPython v1.9.2-443-g236297f4 on 2017-11-06; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> \r\n>>> \

In [ ]:
# main code

In [87]:
%comment %sendtofile main.py
from connectwifi import ipnumber
#from connecthotspot import ipnumber


Scanning wifis: BV6000
I (10212) network: event 1
Connecting to strongest known wifi signal BV6000
.> auth (b0)
> assoc (0)
> run (10)
I (11662) network: event 4
I (12682) network: GOT_IP
Device has ipnumber 192.168.43.6


In [88]:
%comment %sendtofile --append main.py

from webserve_funcs import servestaticfile
from webserve_funcs import servefilelist
from webserve_funcs import readhttpheaders

from websocket_funcs import initiatewebsocket
from websocket_funcs import make_hybi07_header
from websocket_funcs import wswrite
from websocket_funcs import wsread
from websocket_funcs import wsreadprintloop

#from vl6180_funcs import init1, init2, set_reg8, get_reg8, distmm
from ms5611_funcs import setupms5611, aMS5611read


[leftinbuffer] ['']


In [89]:
%comment %sendtofile --append main.py

import uasyncio, ustruct
import machine
from i2cmodule import i2c

# coroutine for handling the write loop (and closing when note comes through thr reader)
async def wswritepulseloop(rpath, w, wscondition):
    rpath = rpath.split(",")
    print(rpath)
    if rpath[0] == "pulse":
        a = machine.ADC(machine.Pin(int(rpath[1])))
        a.atten(a.ATTN_11DB)
        delayms = int(rpath[2])
    elif rpath[0] == "ms5611":
        a = None
        delayms = int(rpath[1])
        
    try:
        n = 1
        delaymsbetweenreadings = 20
        while wscondition[1]:
            while len(wscondition) > 2:
                incoming = wscondition.pop(2)
                print("[WS%d incoming] %s"%(wscondition[0], incoming))
                    
            await w.awrite(make_hybi07_header(n*2))
            for i in range(n):
                if i != 0:
                    await uasyncio.sleep_ms(delaymsbetweenreadings)
                if a is None:
                    x = await aMS5611read(uasyncio)
                    x = int(x+0.5)
                else:
                    x = a.read()
                y = ustruct.pack("BB", (x&0x3F) + 0x30, ((x>>6)&0x3F) + 0x30)
                await w.awrite(y)
            await uasyncio.sleep_ms(delayms)
    except OSError as e:
        print("[WS%d wpexception] %s"%(wscondition[0], str(e)))
        wscondition[1] = 1
    print("[WS%d wpleave]"%wscondition[0])

WScount = 0
async def servewebsocket(rpath, reader, writer, secwebsocketkey):
    global WScount
    await initiatewebsocket(writer, secwebsocketkey)
    await wswrite(writer, "HITHEREws(%d)"%WScount)
    wscondition = [WScount, 1]
    WScount += 1
    loop = uasyncio.get_event_loop()
    loop.create_task(wsreadprintloop(reader, wscondition))
    loop.create_task(wswritepulseloop(rpath, writer, wscondition))
    return False



In [90]:
%comment %sendtofile --append main.py

import os, uasyncio
async def handleconnection(reader, writer):
    receivedrequest = await readhttpheaders(reader)
    rpath = receivedrequest.get("GET")
    
    bcallaclose = True
    if "WebSocketKey" in receivedrequest:
        bcallaclose = await servewebsocket(rpath, reader, writer, receivedrequest["WebSocketKey"])
    elif rpath is None:
        pass
    elif rpath == "":
        await servefilelist(writer)
    else:
        await servestaticfile(writer, rpath)
        
    if bcallaclose:
        await writer.aclose()


In [ ]:
%comment %sendtofile --append main.py

import uasyncio
uasyncio.set_debug(0)

setupms5611()

# enables rewriting handleconnection function without restarting
def handleconnection_indirect(reader, writer):
    try:
        return handleconnection(reader, writer)
    except OSError as e:
        print("handleconnection exception", str(e))
    
port = 80
loop = uasyncio.get_event_loop()
print("* Running on http://%s:%s/" % (ipnumber, port))
loop.create_task(uasyncio.start_server(handleconnection_indirect, ipnumber, port))
loop.run_forever()


* Running on http://192.168.43.6:80/
[] serving file list
.File graphtest.html (7151 bytes)-- served in 176ms
['ms5611', '200']
[WS0] b'Hello'
[WS0 incoming] b'Hello'
..[WS0 close]
[WS0 rlleave]
['pulse', '35', '50']
[WS1] b'Hello'
[WS1 incoming] b'Hello'
[WS0 wpleave]
..[WS1 close]
[WS1 rlleave]
[WS1 wpleave]
['ms5611', '50']
[WS2] b'Hello'
[WS2 incoming] b'Hello'
.[WS2 close]
[WS2 rlleave]
[WS2 wpleave]
['ms5611', '200']
[WS3] b'Hello'
[WS3 incoming] b'Hello'
................................[WS3 close]
[WS3 rlleave]
[WS3 wpleave]
['ms5611', '50']
[WS4] b'Hello'
[WS4 incoming] b'Hello'
....

In [71]:
%sendtofile --source ../sensor_websocket_graph/graphtest.html graphtest.html


Sent 236 lines (7149 bytes).


In [83]:
loop.run_forever()


......

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "/lib/uasyncio/core.py", line 77, in run_forever
  File "/lib/uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 


In [ ]:
%sendtofile --quiet --source ../sensor_websocket_graph/graphtest.html graphtest.html
